In [ ]:
import json
import logging
import os
from pathlib import Path

from wattile.data_processing import prep_for_rnn
from wattile.data_reading import read_dataset_from_file
from wattile.entry_point import init_logging
from wattile.models import ModelFactory

PROJECT_DIRECTORY = Path().resolve().parents[1]
logger = logging.getLogger(str(os.getpid()))

In [ ]:
"""
define parametric space
"""
parametric_space = [
    {
        "building": "Synthetic Site",
        "targets": [
            "Synthetic Site Electricity Main Total Power"
            # add more targets if necessary
        ],
        "dir_data": "tests/data/Synthetic Site",
    }
    # add more entries if necessary
]

In [ ]:
# read configs file for training
with open(
    PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r"
) as f:
    configs = json.load(f)

In [ ]:
"""
run model training and validation for the entire parametric space
"""
for entry in parametric_space:

    # set building name
    building = entry["building"]

    # set data directory
    data_dir = entry["dir_data"]

    for target in entry["targets"]:

        """
        model training
        """
        logger.info(
            "training for building = {} and target = {}".format(building, target)
        )

        # override configs params based on the parametric space information
        configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / data_dir)
        configs["data_input"]["data_config"] = "{} Config.json".format(building)
        configs["data_input"]["target_var"] = target
        exp_dir = PROJECT_DIRECTORY / building / target
        configs["data_output"]["exp_dir"] = str(exp_dir)

        # create results folder
        init_logging(local_results_dir=configs["data_output"]["exp_dir"])

        # read data
        data = read_dataset_from_file(configs)

        # prepare data for training
        train_df, val_df = prep_for_rnn(configs, data)
        
        # create model
        model = ModelFactory.create_model(configs)
        
        # train model
        results = model.train(train_df, val_df)

        """
        model validation (original data used)
        """
        logger.info(
            "validating for building = {} and target = {}".format(building, target)
        )

        # reset config param for validation
        configs["data_output"]["plot_comparison_portion_start"] = 0.0
        configs["data_output"]["plot_comparison_portion_end"] = 1.0
        configs["learning_algorithm"]["use_case"] = "validation"

        # prepare data for validation
        _, val_df = prep_for_rnn(configs, data)

        # validate model
        results = model.validate(val_df)
